In [1]:
import pickle
from glob import glob

files = sorted(glob('/home/husein/ssd2/processed-youtube-asr-whisper-large-v3/*.pkl'))
files = files[:len(files) // 2]
len(files)

3811

In [2]:
from datasets import Audio
from tqdm import tqdm
import numpy as np
import IPython.display as ipd

In [3]:
sr = 16000
reader = Audio(sampling_rate = sr)

In [4]:
with open(files[0], 'rb') as fopen:
    data = pickle.load(fopen)

In [5]:
max(data[0]['asr_model'][0], key=data[0]['asr_model'][0].get)

'ms'

In [6]:
data[0]

{'wav_data': '/home/husein/ssd2/processed-youtube-asr-whisper-large-v3/#100_WE_FREAKING_HIT_OUR_100TH_EPISODE!__[0Q-Pv3I62x0]/0.mp3',
 'start': 0.0,
 'asr_model': ({'ur': 0.0005483672139234841,
   'ca': 1.7724236158755957e-06,
   'uz': 4.492047196436033e-07,
   'sl': 2.1883515728404745e-05,
   'is': 1.9146273189107887e-06,
   'eu': 1.4504192222375423e-05,
   'lb': 5.434219474409474e-06,
   'tl': 0.0020482514519244432,
   'tk': 1.9805905804304302e-09,
   'ht': 0.0037412315141409636,
   'hr': 7.0581018007942475e-06,
   'uk': 0.0001976174971787259,
   'sq': 1.6484786101500504e-05,
   'tg': 9.537151527183596e-06,
   'he': 0.0001810295070754364,
   'gu': 1.6431578842457384e-05,
   'yi': 5.956028871878516e-06,
   'so': 2.8468155051086796e-06,
   'es': 2.7303754905005917e-05,
   'zh': 0.00011494937643874437,
   'sv': 4.879304105998017e-05,
   'en': 0.005382561590522528,
   'su': 0.00011364376405254006,
   'hy': 3.873963942169212e-05,
   'my': 0.00022876419825479388,
   'de': 0.001561436220072

In [7]:
ms_score = [s['avg_logprob'] for s in data[0]['asr_model'][1]['segments']]
en_score = [s['avg_logprob'] for s in data[0]['asr_model'][2]['segments']]

if np.mean(ms_score) > np.mean(en_score):
    selected = data[0]['asr_model'][1]['segments']
else:
    selected = data[0]['asr_model'][2]['segments']

In [8]:
i = 0

s = selected[i]['start']
e = selected[i]['end']
t = selected[i]['text']
s, e, t

(0.0, 5.0, ' Oh yeah!')

In [9]:
f = data[0]['wav_data']
audio = reader.decode_example(reader.encode_example(f))['array']
y = audio[int(s * sr): int(e * sr)]

In [10]:
ipd.Audio(y, rate = sr)

In [11]:
import malaya_speech

`pyaudio` is not available, `malaya_speech.streaming.pyaudio` is not able to use.


In [12]:
vad = malaya_speech.vad.webrtc()

In [13]:
y_int = malaya_speech.astype.float_to_int(y)
frames_int = list(malaya_speech.utils.generator.frames(y_int, 30, sr))
np.mean([vad(frame) for frame in frames_int])

0.9221556886227545

In [14]:
import malaya
from malaya.text.normalization import cardinal

tokenizer = malaya.tokenizer.Tokenizer(hypen = False, parliament = False, time = False, time_pukul = False,
                                      temperature = False, distance = False, volume = False, duration = False,
                                      weight = False, date = False, money = False)

In [15]:
import unicodedata
import re
import itertools

vocabs = [" ", "a", "e", "n", "i", "t", "o", "u", "s", "k", "r", "l", "h", "d", "m", "g", "y", "b", "p", "w", "c", "f", "j", "v", "z", "0", "1", "x", "2", "q", "5", "3", "4", "6", "9", "8", "7"]

def preprocessing_text(string):
    
    tokenized = tokenizer.tokenize(string)
    string = ' '.join(tokenized)
    string = unicodedata.normalize('NFC', string.lower())
    string = ''.join([c if c in vocabs else ' ' for c in string])
    string = re.sub(r'[ ]+', ' ', string).strip()
    string = (
        ''.join(''.join(s)[:2] for _, s in itertools.groupby(string))
    )
    return string

In [16]:
directory = '/home/husein/ssd3/stt/whisper-v3'

In [17]:
!mkdir {directory}

mkdir: cannot create directory ‘/home/husein/ssd3/stt/whisper-v3’: File exists


In [18]:
import torchaudio
import torch
import os

In [19]:
dataset = []
index = 0
for no, f in enumerate(files):
    
    with open(f, 'rb') as fopen:
        data = pickle.load(fopen)
    
    for i in tqdm(range(len(data))):
        lang = max(data[i]['asr_model'][0], key=data[i]['asr_model'][0].get)
        if lang not in {'ms', 'en'}:
            continue
        
        ms_score = [s['avg_logprob'] for s in data[i]['asr_model'][1]['segments']]
        en_score = [s['avg_logprob'] for s in data[i]['asr_model'][2]['segments']]

        if np.mean(ms_score) > np.mean(en_score):
            selected = data[i]['asr_model'][1]['segments']
        else:
            selected = data[i]['asr_model'][2]['segments']
            
        audio = reader.decode_example(reader.encode_example(data[i]['wav_data']))['array']
        for k in range(len(selected)):
            s = selected[k]['start']
            e = selected[k]['end'] + 0.1
            t = selected[k]['text']
            cleaned_t = preprocessing_text(t)
            if not len(cleaned_t):
                continue
                
            audio_path = os.path.join(directory, f'{index}.mp3')
            a = audio[int(s * sr): int(e * sr)]
            torchaudio.save(audio_path, 
                            torch.tensor(a.astype('float32')).unsqueeze(0), 
                            16000, format='mp3')
            try:
                y_int = malaya_speech.astype.float_to_int(a)
                frames_int = list(malaya_speech.utils.generator.frames(y_int, 30, sr, append_ending_trail = False))

                dataset.append(
                    {
                        'file': audio_path,
                        'text': t,
                        'cleaned': cleaned_t,
                        'no': no,
                        'i': i,
                        'k': k,
                        'avg_logprob': selected[k]['avg_logprob'],
                        'no_speech_prob': selected[k]['no_speech_prob'],
                        'vad': float(np.mean([vad(frame) for frame in frames_int])),
                    }
                )
            except:
                pass
            index += 1

 35%|████████████████████████████████▍                                                           | 36/102 [00:12<00:24,  2.64it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|███████████████████████████████████████████████████████████████████████████████████████████| 115/115 [00:29<00:00,  3.94it/s]


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:04<00:00,  4.65it/s]


 48%|████████████████████████████████████████████▎                                                | 10/21 [00:03<00:03,  2.78it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 41/41 [00:08<00:00,  4.80it/s]


 39%|████████████████████████████████████▋                                                        | 13/33 [00:02<00:04,  4.04it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:02<00:00,  3.34it/s]


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:02<00:00,  2.79it/s]


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  3.57it/s]


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  3.14it/s]


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:04<00:00,  3.94it/s]


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:01<00:00,  3.58it/s]


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:05<00:00,  3.55it/s]


In [20]:
len(dataset)

1141443

In [21]:
dataset[0]

{'file': '/home/husein/ssd3/stt/whisper-v3/0.mp3',
 'text': ' Oh yeah!',
 'cleaned': 'oh yeah',
 'no': 0,
 'i': 0,
 'k': 0,
 'avg_logprob': -0.3756620857153046,
 'no_speech_prob': 0.15171679854393005,
 'vad': 0.9470588235294117}

In [22]:
dataset[-1]

{'file': '/home/husein/ssd3/stt/whisper-v3/1144194.mp3',
 'text': ' Ciao dan Bye Bye',
 'cleaned': 'ciao dan bye bye',
 'no': 3810,
 'i': 18,
 'k': 2,
 'avg_logprob': -0.6422442956404253,
 'no_speech_prob': 0.005675802938640118,
 'vad': 0.0}

In [24]:
import json

with open('dataset-whisper-stt-v3.json', 'w') as fopen:
    json.dump(dataset, fopen)